## Measuring the divergence in empirical and theoretical probabilities of permutations using squared difference

$$
\Delta(a, b) = \sum_{i}(Prob(i, a) - Prob(j, a))^2
$$

In [1]:
import import_ipynb
from Mallows_Notebook import *
from PL_Notebook import *
import metropolis
import math
from tqdm import tqdm_notebook
import itertools

importing Jupyter notebook from Mallows_Notebook.ipynb
importing Jupyter notebook from PL_Notebook.ipynb
PL loaded


In [ ]:
import os
netflix = os.listdir('netflix_soc')
files = ['netflix_soc/' + x for x in netflix]

In [16]:
irish = [#'ED-00001-00000001.soi',\
         'ED-00001-00000001.soi',\
         #'ED-00001-00000003.soi',]
        ]

files = ['irish_soi/' + x for x in irish]

In [ ]:
files = ['ED-00002-00000001.soi',\
         'ED-00002-00000002.soi',\
         'ED-00002-00000003.soi',\
         'ED-00002-00000004.soi',\
         'ED-00002-00000005.soi',\
         'ED-00002-00000006.soi',\
         'ED-00002-00000007.soi']

files = ['preflib_soi/' + x for x in files]

In [17]:
list_of_votes = []
mallows_params = []
pl_params = []

nruns = 10_000

# print(files[0])

print('Projected Time =',120/1000.0 * nruns,'seconds, which is ~', 2/1000.0 * nruns, 'minutes')
for file in tqdm_notebook(files,desc = 'All Files'):
    print(file)
    _, lengths, num_votes, votes = readPreflib.soiInputwithNumVotes(file)
    # print(num_votes)
    # print(list(lengths.keys()))
    list_of_votes.append((num_votes, lengths, votes))
    # print(list_of_votes[0])
    p_mal = runMallows(votes, nruns, lengths, complete=True)
    mallows_params.append(p_mal)
    p_pl = runPL(votes, nruns, lengths,complete=True)
    pl_params.append(p_pl)

Projected Time = 1200.0 seconds, which is ~ 20.0 minutes


irish_soi/ED-00001-00000001.soi


In [19]:
mallows_params, pl_params
pickle.dump([mallows_params, pl_params], open('pickle/ireland_00000001soi_params.p','wb'))
# prob pickle things here
mallows_params, pl_params

([[array([1, 2, 3]), 0.026104278367001077]],
 [array([0.03406422, 0.07380935, 0.27913067, 0.18167261, 0.13855618,
         0.08574646, 0.04970188, 0.03203864, 0.01561149, 0.01538392,
         0.01479098, 0.0794936 ])])

In [11]:
def all_possible_votes(n):
    all_votes = []
    for i in range(1,n+1):
        # print(i)
        sub = list(itertools.permutations(range(1,n+1),i))
        #sub = itertools.permutations(range(1,i+1))
        for p in sub:
            all_votes.append(p)
    return all_votes
import sys
sys.getsizeof(all_possible_votes(10))

81528056

In [12]:
_,_,temp = list_of_votes[0]
known = [tuple(x) for x in list(zip(*temp))[1]]
pos = all_possible_votes(4)
len(pos)
# [x for x in pos if x not in known]
# len(known)

64

In [20]:
def squaredDif(a, b):
    return (a - b)**2

table = []
col_names = ['Number_Votes','Mallow\'s_Divergence','Plackett-Luce_Divergence']

for i in tqdm_notebook(range(len(list_of_votes))):
    num_votes, lengths, votes = list_of_votes[i]
    num_alternatives = len(lengths)
    sigma, phi = mallows_params[i]
    pl_weights = pl_params[i]
    
    delta_mallows = 0
    delta_plackett = 0
       
    freq_list = list(zip(*votes))[0]
    known_votes = [tuple(x) for x in list(zip(*votes))[1]]
    # possibles = all_possible_votes(num_alternatives)
    # empirical_zero = [x for x in possibles if x not in known_votes]
    
    for vote in tqdm_notebook(all_possible_votes(num_alternatives)):
        if (vote in known_votes):
            index = known_votes.index(vote)
            num_occurances = freq_list[index]
            
            empirical = num_occurances / num_votes
            mallows = mallowsProb(vote, sigma, phi)
            plackett = probPlackett(vote, pl_weights)
            delta_mallows += squaredDif(mallows, empirical)
            delta_plackett += squaredDif(plackett, empirical)
            # print(vote)
        else:
            empirical = 0
            mallows = mallowsProb(vote, sigma, phi)
            plackett = probPlackett(vote, pl_weights)
            delta_mallows += squaredDif(mallows, empirical)
            delta_plackett += squaredDif(plackett, empirical)
    
#     for entry in votes:
#         num_occurances, vote = entry
#         empirical = num_occurances / num_votes
#         mallows = mallowsProb(vote, sigma, phi)
#         plackett = probPlackett(vote, pl_weights)
#         delta_mallows += squaredDif(mallows, empirical)
#         delta_plackett += squaredDif(plackett, empirical)
    
    table.append([num_votes, delta_mallows, delta_plackett])

MemoryError: 

In [14]:
import pandas as pd

npdata = np.array(table)
results_df = pd.DataFrame(data=npdata,columns=col_names)
results_df.index += 1
# pickle.dump(results_df, open('pickle/divergence10k.p','wb'))
results_df

,Number_Votes,Mallow's_Divergence,Plackett-Luce_Divergence
1,29988.0,16145.991197,64.609649


In [ ]:
perms = list(itertools.permutations([1,2,3]))
print(perms)
(1,2,3) in perms

In [15]:
pickle.dump([mallows_params, pl_params, table], open('./pickle/ireland_0001_table_10k.p', 'wb'))

In [ ]:
mallows_params

In [ ]:
pl_params